In [ ]:
from atproto import Client, client_utils
# import dataset
from nwsapy import api_connector

import datetime
import json
import os
# import pickle
from pprint import pp as prettyprint
from time import sleep

In [ ]:
api_connector.set_user_agent('NotNWSTornado.bsky.social', 'Mike Stucka - last name at that Google mail service')

In [ ]:
timestamp = datetime.datetime.now()
naptime = 2

In [ ]:
weatherskyconfigfile = "weathersky-config.json"
if not os.path.exists(weatherskyconfigfile):
    print("You're missing a weathersky-config.json file. Make one from the template.")
else:
    with open(weatherskyconfigfile, "r", encoding="utf-8") as infile:
        config = json.load(infile)

In [ ]:
recentspottingsfile = "recent-spottings.json"
if not os.path.exists(recentspottingsfile):
    print(f"No {recentspottingsfile} found. Intializing.")
    recentspottings = {}
else:
    with open(recentspottingsfile, "r", encoding="utf-8") as infile:
        recentspottings = json.load(infile)
        print(f"{recentspottingsfile} found with {len(recentspottings):,} entries.")

In [ ]:
tor_warnings = api_connector.get_active_alerts(event = "Tornado Warning")

In [ ]:
if tor_warnings.has_any_request_errors:
    raise ConnectionError(f"Details: {tor_warnings['detail']}")

In [ ]:
new_warnings = []
for tor_warning in tor_warnings:
    if tor_warning.id in recentspottings:
        print(f"Already seen {tor_warning.id}")
    else:
        recentspottings[tor_warning.id] = {
            "first_seen": json.dumps(timestamp, default=str),
            "entryholder": json.dumps(tor_warning.to_dict(), default=str)
        }
        new_warnings.append(tor_warning)

In [ ]:
def clean_text(incoming):
    result = incoming.replace("*", " ").replace("\n", " ").replace("  ", " ").replace("  ", " ").strip()
    return result

In [ ]:
def extract_description(description):        
    """Key text seems to be found before HAZARD, in the last section before a double newline"""
    try:
        chunks = description.split("\n\n")
        for i, chunk in enumerate(chunks):
            if "HAZARD" in chunk:
                # print(f"Hazard found in chunk {i}")
                break
        if i == 0:
            # print("Returning description")
            return description
        else:
            # print("success!")
            return clean_text(chunks[i-1])
    except:
        # print("except")
        return description

In [ ]:
new_narratives = {}
for configentry in config:
    # If this matches the region and quality we're looking for
    local_narratives = []
    for warning in new_warnings:
        narrative = f"{'; '.join(warning.area_desc)}: {extract_description(warning.description)}. "
        if "CMAMtext" in warning.parameters:
            narrative += f"{' '.join(warning.parameters['CMAMtext'])}"
        print(narrative)
        local_narratives.append(narrative)
    new_narratives[configentry] = local_narratives

In [ ]:
for configentry in config:
    if configentry in new_narratives and len(new_narratives[configentry]) > 0:    # If we have stuff to report
        client = Client()
        client.login(f"{configentry}.bsky.social", config[configentry]['app_password'])
        for narrative in new_narratives[configentry]:
            post = client.send_post(narrative)
            sleep(naptime)

In [ ]:
with open(recentspottingsfile, "w", encoding="utf-8") as outfile:
    outfile.write(json.dumps(recentspottings, indent=4*' ', default=str))